# <font color='red' style='font-size: 30px;'> Rede Complexa Baseada em CDR's </font>
<hr style='border: 2px solid red;'>

## <font color = 'black' style='font-size: 26px;'> Imports </font>  
<hr style = 'border: 1.5px solid black;'>

In [3]:
# Lista de Imports
import pandas as pd
import numpy as np
import networkx as nx
import time
from functions import save_with_pickle, load_with_pickle

# Variáveis Globais
EXECUTAR = True

## <font color = 'black' style='font-size: 26px;'> Importando os Dados </font>  
<hr style = 'border: 1.5px solid black;'>

In [5]:
# Importação dos Dados
dataset = pd.read_csv('../Dados/cdr_processed_Uberlândia.csv', sep=',')

## <font color = 'black' style='font-size: 26px;'> Realizando o Pré-Processamento </font>  
<hr style = 'border: 1.5px solid black;'>

In [7]:
# Selecionando as ligações com duração maiores que 15 segundos
dataset = dataset[dataset['Duração'] >= 0.24]
dataset.shape

(7618927, 6)

In [8]:
len(dataset['Data'].unique())

30

In [9]:
df = dataset.groupby(['ID_Origem', 'ID_Destino']).size().reset_index(name='Count')
df = df.query("Count >= 5")
df.head(3)

,ID_Origem,ID_Destino,Count
0,0000489E396F11608F0DF012E4FAB9B8,07D92D9481B1C217EC7B30E2504140D6,15
1,0000489E396F11608F0DF012E4FAB9B8,1AA438F6F9D834D57A83AC905DD99DE6,7
2,0000489E396F11608F0DF012E4FAB9B8,4A1C09A9877D94ECF35B0320EE94364D,7


In [10]:
agrup_origem = df.groupby('ID_Origem').sum().reset_index()
agrup_origem = agrup_origem.query("Count <= 200")['ID_Origem']

In [11]:
agrup_destino = df.groupby('ID_Destino').sum().reset_index()
agrup_destino = agrup_destino.query("Count <= 200")['ID_Destino']

In [12]:
# Todos os IDs que fizeram ou receberam mais de 6 ligações
nos = np.unique(np.append(agrup_origem, agrup_destino))
save_with_pickle(nos, '../Dados/Processados/', 'nos')
len(nos)

372045

In [13]:
df = df.query("ID_Origem in @nos | ID_Destino in @nos")
df.shape

(551748, 3)

In [14]:
if EXECUTAR:    
    # Instanciando o dicionário
    users = {}
    for ID in df['ID_Origem'].unique():
        users[ID] = {'Ligações': list()}
        
    # Agrupando as ligações em um dicionário    
    for _, row in df.iterrows():
        users[row['ID_Origem']]['Ligações'].append(row['ID_Destino'])
    
    # Salva o dicionário em um objeto pickle
    save_with_pickle(users, '../Dados/Processados/', 'dict_dados')
    
else:
    # Carrega o dicionário em um objeto pickle
    users = load_with_pickle('../Dados/Processados/', 'dict_dados')

## <font color = 'black' style='font-size: 26px;'> Criando a Rede </font>  
<hr style = 'border: 1.5px solid black;'>

In [16]:
def contabiliza_ligacoes(origem, destino, dicionario):
    total = 0
    if origem in dicionario:
        total += dicionario[origem]['Ligações'].count(destino)
        
    if destino in dicionario:
        total += dicionario[destino]['Ligações'].count(origem)
        
    return total    

In [17]:
def inicializa_grafo(nos):
    grafo = nx.Graph() 
    for i in nos:
        grafo.add_node(i)
    return grafo

In [18]:
def cria_arestas(grafo, nos, dicionario):
    
    # Cria arestas entre todos os nós da lista
    for i in dicionario.keys():
        for j in dicionario[i]['Ligações']:
            if not grafo.has_edge(i, j):
                grafo.add_edge(i, j)
    return grafo

In [19]:
def gera_grafo(nos, dicionario):    
    grafo = inicializa_grafo(nos)
    grafo = cria_arestas(grafo, nos, dicionario)
    return grafo

In [20]:
##### grafo = gera_grafo(nos, users)
# nx.write_graphml(grafo, "cdr_grafo.graphml")